In [ ]:
# preparation database
import time
import sqlite3

dbpath = 'index.db'
dbconn = sqlite3.connect(dbpath)
dbcur = dbconn.cursor()

path = 'cluster.db'
conn = sqlite3.connect(path)
cur = conn.cursor()

In [ ]:
def create_cluster_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Cluster (
                     addr INTEGER PRIMARY KEY,
                     cluster NOT NULL);
                ''')

def insert_clusters(clusters):
    cur.executemany('''INSERT OR IGNORE INTO Cluster (
                       addr, cluster) VALUES (
                       ?, ?);
                    ''', clusters)

def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    
def journal_mode(mode):
    sql = f'PRAGMA journal_mode = {mode}'
    cur.execute(sql)
    conn.commit()

def synchronous(mode):
    sql = f'PRAGMA synchronous = {mode}'
    cur.execute(sql)
    conn.commit()
    
def get_cluster_id_(f, t):
    dbcur.execute('''SELECT id FROM TxID WHERE
                     id BETWEEN ? AND ? 
                     ORDER BY id ASC;''', (f, t))
    return dbcur.fetchall()

def get_cluster_id(f, t):
    dbcur.execute('''SELECT id, -1 FROM TxID WHERE
                     id BETWEEN ? AND ? 
                     ORDER BY id ASC;''', (f, t))
    return dbcur.fetchall()

def get_max_id():
    dbcur.execute('''SELECT MAX(id) FROM TxID;''')
    return dbcur.fetchone()[0]

In [ ]:
def init_cluster(record):
    return record[0], -1

In [ ]:
create_cluster_table()

In [ ]:
sid = 1
eid = int(get_max_id())
term = 10000000
print(f'Start from {sid} to {eid}')

try:
    synchronous('NORMAL')
    journal_mode('WAL')
    for s, e in zip(range(sid, eid, term), 
                    range(sid+term, eid+term, term)):
        stime = time.time()
        begin_transactions()
        if e < eid:
            e = e-1
        data = list()
        insert_clusters(get_cluster_id(s, e))
        commit_transactions()
        etime = time.time()
        print(f'Copy Job done from {s} to {e} during {etime-stime}')
except KeyboardInterrupt:
    print(f'KeyboardInterrupt detected. Commit transactions.')
    try:
        commit_transactions()
    except sqlite3.OperationalError:
        pass
finally:
    try:
        commit_transactions()
    except sqlite3.OperationalError:
        pass
    dbconn.close()
    conn.close()